# Experimental Sampling Pipeline

This notebook demonstrates the complete pipeline from sampling model generations and creating matched random mutants for experimental validation:
1. Loading model generations and BLAST results (computed via respective scripts)
2. Computing sequence similarities
3. Sampling sequences based on similarity brackets
4. Generating matched random mutants

## Inputs
Before using this notebook, run `scripts/generate_sequences.py` to generate sequences, and then `scripts/blast_script.sh` to blast those sequences against the training data. The resulting data will be used in this notebook.

## Setup
Install required packages:

In [ ]:
!pip install biopython pandas numpy

In [ ]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

from origen.alignment import compute_sequence_similarity
from origen.random_mutants import generate_matched_random_mutant
from origen.tokenizers import extract_oriv

## 1. Load and Preprocess Data

Load generated sequences and BLAST results:

In [ ]:
# Load training data
from datasets import load_dataset

train_dataset = load_dataset('jirvine/origen-replicons-doric-and-plsdb')["train"]

# Create mapping from sequence IDs to sequences
training_orivs = train_dataset['oriv_sequence']
training_orivs_idx = {}
for i, oriv in enumerate(training_orivs):
   training_orivs_idx['seq' + str(i)] = oriv # zero-indexed

# Load generated sequences
with open('path/to/generated_seqs.txt', 'r') as f:
    generations = f.readlines()

# Process generated sequences
generated_orivs_idx = {}
for i, generation in enumerate(generations):
    generated_oriv = extract_oriv(generation)
    generated_orivs_idx[f'seq{i+1}'] = generated_oriv  # 1-indexed

# Load BLAST results
blast_hits = pd.read_csv('path/to/blast_hits.tsv', sep='\t')

# Add sequences and lengths to blast hits
blast_hits['query_seq'] = blast_hits['query_id'].map(generated_orivs_idx)
blast_hits['subject_seq'] = blast_hits['subject_id'].map(training_orivs_idx)
blast_hits['query_length'] = blast_hits['query_seq'].str.len()
blast_hits['subject_length'] = blast_hits['subject_seq'].str.len()

## 2. Compute Sequence Similarities

Calculate similarity scores for all BLAST hits:

In [ ]:
# Calculate similarities using needle alignment
blast_hits['needle'] = blast_hits.apply(
    lambda row: compute_sequence_similarity(row['query_seq'], row['subject_seq']),
    axis=1
)

# Filter by length difference, to avoid low similarity due simply to length differences
LENGTH_DIFF_THRESHOLD = 10
blast_hits_filtered = blast_hits[
    (blast_hits['query_length'] >= blast_hits['subject_length'] - LENGTH_DIFF_THRESHOLD) &
    (blast_hits['query_length'] <= blast_hits['subject_length'] + LENGTH_DIFF_THRESHOLD)
]

## 3. Sample by Similarity Brackets

Group sequences into similarity brackets and sample representatives:

In [ ]:
# Define similarity brackets
brackets = [
    (99, 100), (98, 99), (97, 98), (96, 97), (95, 96), (94, 95), (93, 94),
    (92, 93), (91, 92), (90, 91), (89, 90), (88, 89), (87, 88), (86, 87),
    (85, 86), (84, 85), (83, 84), (82, 83), (81, 82), (80, 81)
]

# Sample sequences from each bracket
sampled_sequences = []
for min_needle, max_needle in brackets:
    bracket_df = blast_hits_filtered[
        (100 * blast_hits_filtered['needle'] >= min_needle) & 
        (100 * blast_hits_filtered['needle'] < max_needle)
    ]
    if not bracket_df.empty:
        sample = bracket_df.sample(n=1, random_state=42)
        sampled_sequences.append({
            'bracket': f'{min_needle}-{max_needle}',
            'query_seq': sample['query_seq'].iloc[0],
            'subject_seq': sample['subject_seq'].iloc[0],
            'similarity': sample['needle'].iloc[0]
        })

## 4. Generate Matched Random Mutants

For each sampled sequence pair, generate a matched random mutant:

In [ ]:
# Generate matched random mutants
for seq in sampled_sequences:
    random_mutant = generate_matched_random_mutant(
        seq['query_seq'],
        seq['subject_seq']
    )
    seq['random_mutant'] = random_mutant
    seq['random_mutant_similarity'] = compute_sequence_similarity(
        random_mutant,
        seq['subject_seq']
    )

# Convert to DataFrame
results_df = pd.DataFrame(sampled_sequences)
print("Sampling and random mutant generation complete!")
results_df.head()